# Purpose

Arrange a script to calculate a Fourier modulus

## install

In [ ]:
%pip install -q git+https://github.com/Surpris/BraggPy.git

## setup modules

In [ ]:
import h5py
import json
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from matplotlib.colors import LogNorm
import os
import braggpy
import time

%matplotlib inline

In [ ]:
def print_h5_tree(val, pre=''):
    if isinstance(val, h5py._hl.files.File):
        print(val.filename)

    items = len(val)
    for key, val in val.items():
        items -= 1
        if items == 0:
            # the last item
            if isinstance(val, h5py._hl.group.Group):
                print(pre + '└── ' + key)
                print_h5_tree(val, pre+'    ')
            else:
                print(pre + '└── ' + key)
        else:
            if isinstance(val, h5py._hl.group.Group):
                print(pre + '├── ' + key)
                print_h5_tree(val, pre+'│   ')
            else:
                print(pre + '├── ' + key)

In [ ]:
def load_input_parameter(fpath: str) -> dict:
    with open(fpath, 'r') as ff:
        return json.loads(ff.read())

In [ ]:
def generate_lattice_coordinates(parameters: dict) -> np.ndarray:
    prev_number_of_atoms: int = 0
    coors_origin: np.ndarray = None
    coors_inside: np.ndarray = None
    target = parameters.get('target')
    for ind_max in range(10, 50):
        coors_origin = braggpy.make_lattice_points(
            target['unit_cell_length']['value'],
            lattice_type=target['crystal_structure']['value'],
            ind_min=-ind_max, ind_max=ind_max
        )
        coors_inside = braggpy.is_inside(
            coors_origin,
            target['crystal_characteristic_length']['value'],
            target['crystal_shape']['value']
        )
        if prev_number_of_atoms == coors_inside.shape[0]:
            break
        prev_number_of_atoms = coors_inside.shape[0]
    return coors_inside

In [ ]:
def generate_momentum(parameters: dict) -> dict:
    hv: float = None
    if parameters['incident_xray_beam'].get('photon_energy'):
        hv = parameters['incident_xray_beam'].get('photon_energy')['value']
    else:
        hv = 12.3849 / parameters['incident_xray_beam'].get('wavelength')['value']
    reciprocal_lattice = parameters['momentum_space']
    return braggpy.generate_momentum(
        hv,
        reciprocal_lattice['momentum_max']['value'],
        reciprocal_lattice['momentum_step']['value'],
    )

In [ ]:
def calculate_euler_angle(parameters: dict, momentum_coors: dict) -> np.ndarray:
    target = parameters['target']
    euler = np.rad2deg(braggpy.calc_euler_hkl(
        target['unit_cell_length']['value'],
        momentum_coors['k_0'],
        *target['miller_index']['value']
    ))
    return euler

In [ ]:
def add_params(group, key, values):
    if values.get('value'):
        _ = group.create_dataset(key, data=values['value'])
        return group
    group_ = group.create_group(f'{key}')
    for key_, values_ in values.items():
        group_ = add_params(group_, key_, values_)
    return group


def save_results(fpath, parameters, euler, momentum_coors, fourier_modulus, coors):
    with h5py.File(fpath, 'w') as tree:
        # add input parameters
        tree = add_params(tree, 'input_parameters', parameters)

        # add output
        group = tree.create_group('outputs')
        sub_group = group.create_group('target')
        _ = sub_group.create_dataset('euler_angle', data=euler)

        sub_group = group.create_group('momentum_space')
        _ = sub_group.create_dataset('momentum_x', data=momentum_coors['qxx'])
        _ = sub_group.create_dataset('momentum_y', data=momentum_coors['qyy'])
        _ = sub_group.create_dataset('momentum_z', data=momentum_coors['qzz'])

        sub_group = group.create_group('incident_xray_beam')
        _ = sub_group.create_dataset('wavenumber', data=momentum_coors['k_0'])

        sub_group = group.create_group('main_outputs')
        _ = sub_group.create_dataset('fourier_modulus', data=fourier_modulus)
        _ = sub_group.create_dataset('coordinates_of_atoms', data=coors)

## directory setting

In [ ]:
outputdir = '../output_data/01_simulation'

if not os.path.exists(outputdir):
    os.makedirs(outputdir)

## calculation with `param.json`

In [ ]:
%%time

n_workers: int = 4  # for parallel calculation of Bragg reflection

inputpath = '../input_data/param.json'
fname = 'fourier_modulus_R{0:.2f}A.h5'

props = np.arange(1.0, 0.0, -0.1)

# load parameters
parameters: dict = load_input_parameter(inputpath)
R0: float = parameters['target']['crystal_characteristic_length']['value'] * 1.0

# generate momentum coordinates
momentum_coors = generate_momentum(parameters)

# main loop
st = time.time()
for p in props:
    # update characteristic length
    parameters['target']['crystal_characteristic_length']['value'] = R = R0 * p
    dstpath = os.path.join(outputdir, fname.format(R))

    # generate lattice coors
    coors_inside = generate_lattice_coordinates(parameters)

    # Euler rotation
    euler = calculate_euler_angle(parameters, momentum_coors)
    coors_euler = braggpy.euler_rotate(
        coors_inside, euler, 1
    )
    print(dstpath, R, coors_euler.shape)

    # calculate modulus
    F = braggpy.calc_modulus(
        coors_euler,
        momentum_coors['qxx'], momentum_coors['qyy'], momentum_coors['qzz'],
        n_workers
    )

    # save modulus
    save_results(dstpath, parameters, euler, momentum_coors, F, coors_euler)
    
    print(f"elapsed time: {time.time() - st:.2f} sec.")

## check the outputs

In [ ]:
for fname_ in os.listdir(outputdir):
    print(os.path.join(outputdir, fname_))

In [ ]:
output_file_name_list = os.listdir(outputdir)
with h5py.File(os.path.join(outputdir, output_file_name_list[0]), "r") as tree:
    print_h5_tree(tree)